# Função para predição do valor de fechamento 

In [61]:
"""
Exemplo de dados_entrada:
dados_entrada = {
    "Date": "2024-11-27",
    "Open": 190.75,
    "High": 195.10,
    "Low": 189.50,
    "Volume": 72000000
}
"""

import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import pickle, joblib

modelo_hiperp = torch.load('modelo_aapl_lstm.pth', weights_only=False)
modelo = modelo_hiperp['model_state_dict']
hiperp = modelo_hiperp['hyperparameters']
print(hiperp)

input_size = hiperp['input_size']
hidden_size = hiperp['hidden_size']
num_layers = hiperp['num_layers']
learning_rate = hiperp['learning_rate']
batch_size = hiperp['batch_size']
num_epochs = hiperp['num_epochs']
output_size = hiperp['output_size']
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)
        self.lstm2 = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm1(x, (h0, c0))
        out = self.fc1(out[:, -1, :])
        out, _ = self.lstm2(x, (h0, c0))
        out = self.fc2(out[:, -1, :])
        return out
    
    
def valor_fechamento(dados_entrada):
    
    model = LSTM(input_size, hidden_size, num_layers, output_size).to(device)
    model.load_state_dict(modelo)
    model.eval()
    scaler = joblib.load('scaler.pkl')

    
    # estabelecimento de DF com os dados de entrada, normalização e criação das features extras
    df = pd.DataFrame([dados_entrada])
    df['Date'] = pd.to_datetime(df['Date'])
    df['Weekday'] = df['Date'].dt.weekday
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['day_sin'] = np.sin(2 * np.pi * df['Date'].dt.dayofyear / 365)
    df['day_cos'] = np.cos(2 * np.pi * df['Date'].dt.dayofyear / 365)
    df['Close'] = 0  # necessário para normalizar os dados
    
    cols_norm = ['Open', 'High', 'Low', 'Close', 'Volume', 'Weekday', 'Month', 'Year']
    df_norm = df[cols_norm + ['day_sin', 'day_cos']].copy()
    df_norm[cols_norm] = scaler.transform(df_norm[cols_norm])
    df_norm = df_norm.drop('Close', axis=1).copy()
    
    # predição de Close
    X = torch.tensor(df_norm.values, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        df_norm['Close'] = float(model(X).item())
    
    # desnormalizar o valor de Close
    # min_close, max_close = scaler.data_min_[3], scaler.data_max_[3]  # 'Close' é o índice 3
    # predicao_real = predicao_normalizada * (max_close - min_close) + min_close
    df_norm2 = scaler.inverse_transform(df_norm[cols_norm].copy())  # acertar a ordem das colunas e desnormalizar
    
    return round(df_norm2[0][3], 4)


{'input_size': 9, 'hidden_size': 64, 'num_layers': 1, 'num_epochs': 46, 'batch_size': 128, 'output_size': 1, 'learning_rate': 0.00864523171136962}


### Teste

In [62]:
dados_entrada = {
    "Date": "2024-01-02",
    "Open": 187.149994,
    "High": 188.440002,
    "Low": 183.889999,
    "Volume": 82488700
}
vlr = valor_fechamento(dados_entrada)

print(f'Valor previsto de fechamento: {round(vlr, 2)}')


Valor previsto de fechamento: 165.61


#### Comparação com informações reais

In [63]:
from base_historica.base_historica_AAPL import download_base
download_base('AAPL', "2024-01-02", "2024-01-03", 'base_teste.csv')
df_teste = pd.read_csv('base_teste.csv')
df_teste.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2024-01-02,187.149994,188.440002,183.889999,185.639999,184.734985,82488700
